# Input 
Input will be done with (objects+1) x (member+1) table, where each row contains an object and it's ranking. First row has the name for the members (+ the "type" of objects)
This can be use as an experiment for the typing recently introduced in Python

In [1]:
from typing import Union
import numpy as np
import numpy.typing as npt
from IPython.display import Markdown, display
def printWithMarkdown(string):
    display(Markdown(string))
 
InputRanking: npt.ArrayLike = np.array([
    ["Game",                            "Alessio",  "Marco",    "Andrea"],
    ["Ghostrunner",                     5,          3,          2],
    ["Monster Sanctuary",               2,          2,          6],
    ["Naruto to Boruto: Shinobi Striker", 1,         8,          1],
    ["Killsquad",                       3,          5,          3],
    ["Destroy All Humans!",             4,          4,          4],
    ["Chicken Police",                  8,          1,          8],
    ["Rougue Heroes",                   6,          6,          5],
    ["Suzerain",                        7,          7,          7]
])

In [2]:
import pandas as pd

dfRanking: pd.DataFrame = pd.DataFrame(
    InputRanking[1:, 1:], index=InputRanking[1:, 0], columns=InputRanking[0, 1:])
dfRanking


,Alessio,Marco,Andrea
Ghostrunner,5,3,2
Monster Sanctuary,2,2,6
Naruto to Boruto: Shinobi Striker,1,8,1
Killsquad,3,5,3
Destroy All Humans!,4,4,4
Chicken Police,8,1,8
Rougue Heroes,6,6,5
Suzerain,7,7,7


# State management
Data will be processed using "Rules", formed by a "condition" that will be verified and an "application".
State is formed by the "rankings" and the "assigned objects". 

In [3]:
from copy import deepcopy
from IPython.display import display
from typing import Tuple

# Fills list
def padList (inputList:list, length:int, padWith=""):
    return inputList + [padWith]*max(length - len(inputList), 0)

# Generates all possible combinations
def numpy_combinations(x): 
    idx = np.stack(np.triu_indices(len(x), k=1), axis=-1)
    return x[idx]

class Division:
    objects: npt.ArrayLike
    members: npt.ArrayLike
    objectType: str
    rankings: npt.ArrayLike
    divisions: dict[str, list[str]]
    counts: npt.ArrayLike
    toAssign: npt.ArrayLike
    enabledPriority: npt.ArrayLike

    #Initializes it all
    def __init__(self, matrix: np.array = [["ObjectType", "Object"], ["ObjectA", 1]], toAssign=None):
        self.objects = matrix[1:, 0]
        self.members = matrix[0, 1:]
        self.objectType = matrix[0, 0]
        self.rankings = matrix[1:, 1:].astype(np.int8)
        self.divisions = {member: [] for member in self.members} # Current Division
        self.counts = np.zeros(self.members.shape, dtype=np.int8) # Current Counts for distributed objects
        if(toAssign==None):
            obj_per_member = np.floor(self.objects.shape[0]/self.members.shape[0])
            remaining = self.objects.shape[0] - self.members.shape[0]*obj_per_member
            self.toAssign = np.array([obj_per_member+remaining]+(self.members.shape[0]-1)*[obj_per_member])
        self.enabledPriority = np.ones_like(self.members, dtype=np.int8) # Whether priority is enabled
    def allDivided(self)->bool:
        return self.objects.shape[0] == 0
    def remaining(self):
        return self.toAssign - self.counts
    def print(self):
        maxAssigned = np.max(self.counts)
        with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
            printWithMarkdown(pd.DataFrame(self.rankings, index=self.objects, columns=self.members).to_markdown())
            printWithMarkdown(pd.DataFrame({member:padList(inputList=self.divisions[member],length=maxAssigned,padWith="") for member in self.members}).to_markdown())

    def getPrioritized(self):
        return (self.rankings <= self.remaining()*self.enabledPriority)

    def firstConflict(self, checkEnoughRemaining=True) -> Tuple[str,npt.ArrayLike]:
        # Take first conflict after ordering
        self.order()
        prioritized = self.getPrioritized()
        conflicts = np.argwhere(prioritized.sum(axis=1)>=2).flatten()
        remainingGames = self.remaining()
        # Make a list of members in conflict for each member
        isInConflict = np.zeros((self.members.shape[0],self.members.shape[0]))
        for i in conflicts:
            membersInConflict = np.argwhere(prioritized[i]==True).flatten()
            for (mA, mB) in numpy_combinations(membersInConflict):
                isInConflict[mA,mB] = 1
                isInConflict[mB,mA] = 1

        for i in conflicts:
            enoughRemainingGames = True
            membersInConflict = np.argwhere(np.sum(prioritized[i,:]*isInConflict, axis=1)>0)
            # And see whether there are enough remaining games
            for j in membersInConflict:
                if(remainingGames[j]<=1):
                    enoughRemainingGames = False | (not checkEnoughRemaining)
                    break
            if (enoughRemainingGames):
                return (self.objects[i], np.argwhere(prioritized[i]==True).flatten())
        return ()

    def order(self):
        summedPriority = self.rankings.sum(axis=1)
        reordered = np.lexsort((self.objects, summedPriority))
        self.objects = self.objects[reordered]
        self.rankings = self.rankings[reordered,:]
    
    def disablePriority(self, member):
        index = np.argwhere(self.members==member)[0][0]
        self.enabledPriority[index]=0

    def assignObject(self, assignedObject: str, receiver: str):
        objIndex = np.argwhere(self.objects == assignedObject)[0][0]
        deletedRankings = self.rankings[objIndex, :]
        newState: Division = deepcopy(self)
        newState.objectType = self.objectType
        newState.objects = np.delete(self.objects, obj=objIndex)
        newState.rankings = np.delete(self.rankings, axis=0, obj=objIndex)
        newState.divisions[receiver].append(assignedObject)
        index = np.argwhere(newState.members==receiver)[0][0]
        newState.counts[index] = newState.counts[index] + 1
        newState.rankings = newState.rankings - 1 * \
            (newState.rankings > np.tile(
                deletedRankings, (newState.objects.shape[0], 1)))
        newState.enabledPriority = np.ones_like(self.members, dtype=np.int8)
        return newState


In [4]:
from typing import  Union
# this is basically an abstract class
class Rule:
    # constructor
    def __init__(self):
        pass

    # checkConditions takes currentState and returns true or false
    def checkConditions(self, state: Division) -> bool:
        return false

    # applies rules
    def apply(self, state: Division) -> list[Division]:
        return state

    # refresh rules based on current states
    def refresh(self, states):
        pass

## Forcefully assign Obj_X to Member_A

In [5]:
# Force Assign a object
class ForceAssign(Rule):
    selectedObject:str
    member:str
    executed:bool
    def __init__(self, selectedObject: str, member: str):
        self.selectedObject = selectedObject
        self.member = member
        self.executed = False

    def checkConditions(self, state: Division) -> bool:
        if(not self.executed):
            return ((np.isin(state.members, element=self.member, assume_unique=True)) &
                    (np.isin(state.objects, element=self.selectedObject, assume_unique=True)))
        else:
            return False

    def apply(self, state: Division) -> list[Division]:
        self.executed = True
        return [state.assignObject(self.selectedObject, self.member)]

    def refresh(self, states):
        if(len(states)==1):
            self.executed = False


In [6]:
forceAssignKillsquad: Rule = ForceAssign("Killsquad", "Marco")
state = Division(matrix=InputRanking)
newState = forceAssignKillsquad.apply(state)


In [7]:
state.print()
print("Asssigned KillSquad to Marco")
newState[0].print()

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       8 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         8 |       1 |        8 |
| Rougue Heroes                     |         6 |       6 |        5 |
| Suzerain                          |         7 |       7 |        7 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

Asssigned KillSquad to Marco


|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         4 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        5 |
| Naruto to Boruto: Shinobi Striker |         1 |       7 |        1 |
| Destroy All Humans!               |         3 |       4 |        3 |
| Chicken Police                    |         7 |       1 |        7 |
| Rougue Heroes                     |         5 |       5 |        4 |
| Suzerain                          |         6 |       6 |        6 |

|    | Alessio   | Marco     | Andrea   |
|---:|:----------|:----------|:---------|
|  0 |           | Killsquad |          |

## Assign highest ranked object to Member_A

In [8]:
class AssignMostWanted(Rule):
    member:str
    executed:bool
    def __init__(self, member: str):
        self.member = member
        self.executed = False

    def checkConditions(self, state: Division) -> bool:
        if(not self.executed):
            return (np.any(np.isin(element=state.members, test_elements=self.member, assume_unique=True)) &
                    (state.objects.shape[0]>0))
        else:
            return False

    def apply(self, state: Division) -> list[Division]:
        self.executed = True
        memberIndex = np.argwhere(state.members==self.member)[0][0]
        gameIndex = np.argwhere(state.rankings[:,memberIndex]==1)[0][0]
        return [state.assignObject(state.objects[gameIndex], self.member)]

    def refresh(self, states):
        if(len(states)==1):
            self.executed = False


In [9]:
assignMostWanted: Rule = AssignMostWanted("Alessio")
state = Division(matrix=InputRanking)
newState = assignMostWanted.apply(state)

In [10]:
state.print()
print("Asssigned 1st place game for Alessio")
newState[0].print()

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       8 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         8 |       1 |        8 |
| Rougue Heroes                     |         6 |       6 |        5 |
| Suzerain                          |         7 |       7 |        7 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

Asssigned 1st place game for Alessio


|                     |   Alessio |   Marco |   Andrea |
|:--------------------|----------:|--------:|---------:|
| Ghostrunner         |         4 |       3 |        1 |
| Monster Sanctuary   |         1 |       2 |        5 |
| Killsquad           |         2 |       5 |        2 |
| Destroy All Humans! |         3 |       4 |        3 |
| Chicken Police      |         7 |       1 |        7 |
| Rougue Heroes       |         5 |       6 |        4 |
| Suzerain            |         6 |       7 |        6 |

|    | Alessio                           | Marco   | Andrea   |
|---:|:----------------------------------|:--------|:---------|
|  0 | Naruto to Boruto: Shinobi Striker |         |          |

## Assign object without conflict

In [11]:
class AssignNoConflict(Rule):
    upper_bound: npt.ArrayLike

    def __init__(self):
        pass

    def checkConditions(self, state: Division) -> bool:
        prioritized = state.getPrioritized()
        # Conflict happens when prioritized.sum(axis=1) >= 2
        # No Conflict if ==1
        return np.any(prioritized.sum(axis=1) == 1)

    def apply(self, state: Division) -> list[Division]:
        prioritized = state.getPrioritized()
        prioritizedSum = prioritized.sum(axis=1)
        assigned = 0
        for i in range(prioritizedSum.shape[0]):
            if(prioritizedSum[i] == 1):
                state = state.assignObject(
                    state.objects[i-assigned], state.members[np.argwhere(prioritized[i, :] == True)[0, 0]])
                assigned = assigned+1
        return [state]
    
    def refresh(self, states):
        pass

In [12]:
assignNoConflict: Rule = AssignNoConflict()
state = Division(matrix=InputRanking)
newState = assignNoConflict.apply(state)

In [13]:
state.print()
print("Asssigned non prioritized games")
newState[0].print()

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       8 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         8 |       1 |        8 |
| Rougue Heroes                     |         6 |       6 |        5 |
| Suzerain                          |         7 |       7 |        7 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

Asssigned non prioritized games


|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Monster Sanctuary                 |         2 |       1 |        3 |
| Naruto to Boruto: Shinobi Striker |         1 |       4 |        1 |
| Rougue Heroes                     |         3 |       2 |        2 |
| Suzerain                          |         4 |       3 |        4 |

|    | Alessio             | Marco          | Andrea      |
|---:|:--------------------|:---------------|:------------|
|  0 | Killsquad           | Chicken Police | Ghostrunner |
|  1 | Destroy All Humans! |                |             |

## Starting Line Conflict
When there's a conflict between 2 or more members, and all of them have more than one object to Assign, create cases where the conflicted object has been assigned to each

In [14]:
from typing import Tuple
class StartingLineConflict(Rule):
    def __init__(self):
        pass

    def checkConditions(self, state: Division) -> bool:
        return state.firstConflict()!=()
            
    def apply(self, state: Division) -> list[Division]:
        firstConflict, conflictedMembers = state.firstConflict()
        states = [state.assignObject(firstConflict, state.members[i]) for i in conflictedMembers]
        for i in range(len(states)):
            states[i].disablePriority(state.members[conflictedMembers[i]])
        return states

    def refresh(self, states):
        pass

In [15]:
startingLineConflict: Rule = StartingLineConflict()
state = Division(matrix=InputRanking)
newStates = startingLineConflict.apply(state)

In [16]:
state.print()
print("Created two states where game is assigned to different person")
newStates[0].print()
newStates[1].print()

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       8 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         8 |       1 |        8 |
| Rougue Heroes                     |         6 |       6 |        5 |
| Suzerain                          |         7 |       7 |        7 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

Created two states where game is assigned to different person


|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         4 |       2 |        2 |
| Naruto to Boruto: Shinobi Striker |         1 |       7 |        1 |
| Killsquad                         |         2 |       4 |        3 |
| Destroy All Humans!               |         3 |       3 |        4 |
| Chicken Police                    |         7 |       1 |        7 |
| Rougue Heroes                     |         5 |       5 |        5 |
| Suzerain                          |         6 |       6 |        6 |

|    | Alessio           | Marco   | Andrea   |
|---:|:------------------|:--------|:---------|
|  0 | Monster Sanctuary |         |          |

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         4 |       2 |        2 |
| Naruto to Boruto: Shinobi Striker |         1 |       7 |        1 |
| Killsquad                         |         2 |       4 |        3 |
| Destroy All Humans!               |         3 |       3 |        4 |
| Chicken Police                    |         7 |       1 |        7 |
| Rougue Heroes                     |         5 |       5 |        5 |
| Suzerain                          |         6 |       6 |        6 |

|    | Alessio   | Marco             | Andrea   |
|---:|:----------|:------------------|:---------|
|  0 |           | Monster Sanctuary |          |

## X Conflict
there are two object in conflict between the same two member where both have one first and one second place between each

In [17]:
XRanking: npt.ArrayLike = np.array([
    ["Game",                            "Alessio",  "Marco",    "Andrea"],
    ["Ghostrunner",                     1,          2,          4],
    ["Monster Sanctuary",               2,          1,          3],
    ["Naruto to Boruto: Shinobi Striker", 3,         3,          2],
    ["Killsquad",                       4,          4,          1],
    ["KillsquadA",                       5,          5,          5],
    ["KillsquadB",                       6,          6,          6]
])

# "Ghostrunner" and "Monster Sanctuary" are in conflict between Alessio and Marco

In [18]:
class XConflict(Rule):
    def __init__(self):
        pass

    def checkConditions(self, state: Division) -> bool:
        return self.xConflict(state)!=()

    def xConflict(self, state: Division) -> Tuple[Tuple[str,str],Tuple[str,str]]:
        # Check users who have enough games to be assigned. There should be at least 2 for 2 or more members
        remainingGames = state.toAssign - state.counts
        if(remainingGames.sum()<2):
            return ()
        
        # XConflict are conflicting between 2 people, not more
        canBeXConflict = state.getPrioritized().sum(axis=1)==2

        for memberAIndex in np.arange(state.members.shape[0]-1):
            if(remainingGames[memberAIndex]<2):
                break
            memberAFirstTwo = np.argsort(state.rankings[:,memberAIndex])[0:2]
            for memberBIndex in np.arange(1,state.members.shape[0]):
                if(remainingGames[memberBIndex]<2):
                    break
                memberBFirstTwo = np.argsort(state.rankings[:,memberBIndex])[0:2]
                # memberA has (0,1) and memberB has (1,0)
                if (np.any(memberAFirstTwo != memberBFirstTwo[-1::-1])):
                    break
                # last check:
                if np.all(canBeXConflict[memberAFirstTwo]):
                    # Returns the two (object, member) tuples
                    return ((state.objects[memberAFirstTwo[0]],state.members[memberAIndex]),(state.objects[memberBFirstTwo[0]],state.members[memberBIndex]))
        # No X Conflict can be found
        return ()         

    def apply(self, state: Division) -> list[Division]:
        for conflict in self.xConflict(state): 
            state = state.assignObject(conflict[0], conflict[1]) 
        return [state]

    def refresh(self, states):
        pass

In [19]:
xConflict: Rule = XConflict()
xState = Division(matrix=XRanking)
newXState = xConflict.apply(xState)

In [20]:
xState.print()
print("Assigned first place for both")
newXState[0].print()

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         1 |       2 |        4 |
| Monster Sanctuary                 |         2 |       1 |        3 |
| Naruto to Boruto: Shinobi Striker |         3 |       3 |        2 |
| Killsquad                         |         4 |       4 |        1 |
| KillsquadA                        |         5 |       5 |        5 |
| KillsquadB                        |         6 |       6 |        6 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

Assigned first place for both


|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Naruto to Boruto: Shinobi Striker |         1 |       1 |        2 |
| Killsquad                         |         2 |       2 |        1 |
| KillsquadA                        |         3 |       3 |        3 |
| KillsquadB                        |         4 |       4 |        4 |

|    | Alessio     | Marco             | Andrea   |
|---:|:------------|:------------------|:---------|
|  0 | Ghostrunner | Monster Sanctuary |          |

## Assign least wanted object to the one that wants it most based on previous state

In [21]:
class FinishingLineConflict(Rule):
    baseStateIndex:int
    baseState:Division
    def __init__(self, baseStateIndex:int):
        self.baseStateIndex = baseStateIndex
        self.baseState = None
    def checkConditions(self, state: Division) -> bool:
        return ((self.baseState!= None) & (state.firstConflict(checkEnoughRemaining=False)!=()))
            
    def apply(self, state: Division) -> list[Division]:
        #assign least wanted one to the one who has it at an higher ranking in the passed state
        state.order()
        remaining = state.remaining()
        objectToAssign = state.objects[-1]
        indexInBaseState = np.argwhere(self.baseState.objects==objectToAssign)[0][0]
        memberWhoMostWantsIt = np.array([state.members[i] for i in np.argsort(self.baseState.rankings[indexInBaseState,:]) if (remaining[i]>0).any()])
        return [state.assignObject(objectToAssign, memberWhoMostWantsIt[0])]

    def refresh(self, states):
        if((self.baseState== None)&(len(states)>self.baseStateIndex)):
            self.baseState=states[self.baseStateIndex]

In [22]:
state = Division(matrix=InputRanking)
finalConflict: Rule = FinishingLineConflict(0)
finalConflict.refresh([state])
newstate = finalConflict.apply(state)

In [23]:
state.print()
print("Assign game in last position based to highest rank from another state")
newstate[0].print()

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       8 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         8 |       1 |        8 |
| Rougue Heroes                     |         6 |       6 |        5 |
| Suzerain                          |         7 |       7 |        7 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

Assign game in last position based to highest rank from another state


|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       7 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         7 |       1 |        7 |
| Rougue Heroes                     |         6 |       6 |        5 |

|    | Alessio   | Marco   | Andrea   |
|---:|:----------|:--------|:---------|
|  0 | Suzerain  |         |          |

# Processing
We now have a structure for both the input and state, and need to actually use the rules to process our matrix

In [24]:
from queue import SimpleQueue
class RankingProcessor:
    # The various state traversed
    divisionArray:list[Division]
    # the preceding elements
    prec:list[int]
    # the leaf elements
    leafs:list[int]
    # list of rules by priority
    rulesList:list[Rule]
    def __init__(self, startingMatrix:npt.ArrayLike, rulesList:list[Rule]):
        self.divisionArray = [Division(startingMatrix)]
        self.prec = [0]
        self.rulesList = rulesList
        self.leafs=list()
    def execute(self):
        # Restarting
        self.divisionArray = self.divisionArray[0:1]
        self.prec = self.prec[0:1]
        #Initialize Queue
        queue:SimpleQueue = SimpleQueue()
        queue.put(0)
        while(not queue.empty()):
            itemIndex = queue.get()
            ruleApplied = False
            printWithMarkdown("<b>State #"+str(itemIndex)+"</b>"+(self.prec[itemIndex]!=itemIndex)*("( previous state #"+str(self.prec[itemIndex])+")"))
            self.divisionArray[itemIndex].print()
            for rule in self.rulesList:
                rule.refresh(self.divisionArray)
                if(self.divisionArray[itemIndex].allDivided()):
                    break
                if (rule.checkConditions(self.divisionArray[itemIndex])):
                    results = rule.apply(self.divisionArray[itemIndex])
                    for result in results:
                        queue.put(len(self.divisionArray))
                        self.divisionArray.append(result)
                        self.prec.append(itemIndex)
                    printWithMarkdown("<b>applied "+type(rule).__name__+"</b>")
                    ruleApplied = True
                    break
            if(not ruleApplied):
                self.leafs.append(itemIndex)


In [25]:
InputRanking: npt.ArrayLike = np.array([
    ["Game",                            "Alessio",  "Marco",    "Andrea"],
    ["Ghostrunner",                     5,          3,          2],
    ["Monster Sanctuary",               2,          2,          6],
    ["Naruto to Boruto: Shinobi Striker", 1,         8,          1],
    ["Killsquad",                       3,          5,          3],
    ["Destroy All Humans!",             4,          4,          4],
    ["Chicken Police",                  8,          1,          8],
    ["Rougue Heroes",                   6,          6,          5],
    ["Suzerain",                        7,          7,          7]
])

In [26]:
rulesList = [
    AssignMostWanted("Alessio"),
    AssignNoConflict(),
    XConflict(),
    StartingLineConflict(),
    FinishingLineConflict(1)
]

In [27]:
processor = RankingProcessor(InputRanking, rulesList)

In [28]:
processor.execute()

<b>State #0</b>

|                                   |   Alessio |   Marco |   Andrea |
|:----------------------------------|----------:|--------:|---------:|
| Ghostrunner                       |         5 |       3 |        2 |
| Monster Sanctuary                 |         2 |       2 |        6 |
| Naruto to Boruto: Shinobi Striker |         1 |       8 |        1 |
| Killsquad                         |         3 |       5 |        3 |
| Destroy All Humans!               |         4 |       4 |        4 |
| Chicken Police                    |         8 |       1 |        8 |
| Rougue Heroes                     |         6 |       6 |        5 |
| Suzerain                          |         7 |       7 |        7 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

<b>applied AssignMostWanted</b>

<b>State #1</b>( previous state #0)

...

|    | Alessio                           | Marco          | Andrea      |
|---:|:----------------------------------|:---------------|:------------|
|  0 | Naruto to Boruto: Shinobi Striker | Chicken Police | Ghostrunner |
|  1 | Destroy All Humans!               |                | Killsquad   |
|  2 | Suzerain                          |                |             |
|  3 | Rougue Heroes                     |                |             |

<b>applied AssignNoConflict</b>

<b>State #6</b>( previous state #5)

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

|    | Alessio                           | Marco             | Andrea      |
|---:|:----------------------------------|:------------------|:------------|
|  0 | Naruto to Boruto: Shinobi Striker | Chicken Police    | Ghostrunner |
|  1 | Destroy All Humans!               | Monster Sanctuary | Killsquad   |
|  2 | Suzerain                          |                   |             |
|  3 | Rougue Heroes                     |                   |             |

In [29]:
InputRanking: npt.ArrayLike = np.array([
    ["Game",                            "Alessio",  "Marco",    "Andrea"],
    ["Iron Harvest",                    1,          2,          7],
    ["Mafia: Definitive Edition",       10,         1,          1],
    ["Retrowave",                       5,          5,          2],
    ["Between the Stars",               4,          4,          8],
    ["Project Winter",                  3,          3,          10],
    ["Midnight Protocol",               9,          6,          3],
    ["The Henry Stickmin Collection",   7,          7,          5],
    ["Rebel Cops",                      2,          9,          9],
    ["Rustler",                         6,          10,         4],
    ["Farmer’s Dynasty",                8,          8,          6]
])
rulesList = [
    AssignMostWanted("Alessio"),
    AssignNoConflict(),
    XConflict(),
    StartingLineConflict(),
    FinishingLineConflict(1)
]

In [30]:
processor = RankingProcessor(InputRanking, rulesList)
processor.execute()

<b>State #0</b>

|                               |   Alessio |   Marco |   Andrea |
|:------------------------------|----------:|--------:|---------:|
| Iron Harvest                  |         1 |       2 |        7 |
| Mafia: Definitive Edition     |        10 |       1 |        1 |
| Retrowave                     |         5 |       5 |        2 |
| Between the Stars             |         4 |       4 |        8 |
| Project Winter                |         3 |       3 |       10 |
| Midnight Protocol             |         9 |       6 |        3 |
| The Henry Stickmin Collection |         7 |       7 |        5 |
| Rebel Cops                    |         2 |       9 |        9 |
| Rustler                       |         6 |      10 |        4 |
| Farmer’s Dynasty              |         8 |       8 |        6 |

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

<b>applied AssignMostWanted</b>

<b>State #1</b>( previous state #0)

...

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

|    | Alessio        | Marco                         | Andrea            |
|---:|:---------------|:------------------------------|:------------------|
|  0 | Iron Harvest   | Mafia: Definitive Edition     | Retrowave         |
|  1 | Rebel Cops     | Between the Stars             | Midnight Protocol |
|  2 | Project Winter | The Henry Stickmin Collection | Farmer’s Dynasty  |
|  3 | Rustler        |                               |                   |

<b>State #10</b>( previous state #8)

| Alessio   | Marco   | Andrea   |
|-----------|---------|----------|

|    | Alessio           | Marco                         | Andrea            |
|---:|:------------------|:------------------------------|:------------------|
|  0 | Iron Harvest      | Mafia: Definitive Edition     | Retrowave         |
|  1 | Rebel Cops        | Project Winter                | Midnight Protocol |
|  2 | Between the Stars | The Henry Stickmin Collection | Farmer’s Dynasty  |
|  3 | Rustler           |                               |                   |

In [31]:
InputRanking: npt.ArrayLike = np.array([
    ["Game",                            "Alessio",  "Marco",    "Andrea"],
    ["Battletech + DLC",                1,          3,          4],
    ["Sonic Mania",                     2,          1,          1],
    ["The Spiral Scout",                5,          6,          5],
    ["Planet Alpha",                    6,          7,          7],
    ["Override: Mech City Bra",         3,          2,          2],
    ["PUSS!",                           4,          4,          6],
    ["Avernum 3: Ruined World",         7,          5,          3]
])
rulesList = [
    AssignMostWanted("Alessio"),
    AssignNoConflict(),
    XConflict(),
    StartingLineConflict(),
    FinishingLineConflict(1)
]

array([[ True, False, False]])

In [32]:
processor.divisionArray[7].getPrioritized()

array([[ True, False, False]])

In [33]:
processor.divisionArray[7].enabledPriority

array([1, 1, 1], dtype=int8)